### R2R Engine Test
Review how meta data is used to identify the user (`user_id`) and/are enhance the result queries.

####  Engine docs here
https://r2r-docs.sciphi.ai/documentation/python-sdk/ingestion


In [ ]:

from r2r import Document, GenerationConfig, R2R

"""
This is creating 3 entries in `deweylearn_vecs_collection` entries in the database.
https://supabase.com/dashboard/project/wwsgmwylkhzowrfagkuf/editor/88376?schema=vecs

and 1 entry in `document_info_deweylearn_vecs_collection` entry in the database.
https://supabase.com/dashboard/project/wwsgmwylkhzowrfagkuf/editor/88384?schema=public

"""

# Initialize the R2R app
app = R2R()  # You may pass a custom configuration to `R2R`

meta = {
  "title": "Dirks test text",
  "user": "dliebich",
  "entity": "DeweyLearn",
  "user_id": "2bc1f0e5-89ee-4e4b-be75-cdcb22144efb"
}

# Ingest the test document
test_string = """
DeweyLearn is an innovative AI-powered platform designed to assist teachers in analyzing classroom data. 
It processes classroom video recordings, transcribes them using speech-to-text technology, and then applies 
machine learning algorithms to extract key insights. These insights include student engagement, participation 
levels, and teacher feedback patterns. Additionally, DeweyLearn generates actionable recommendations for 
improving teaching strategies and enhancing student learning outcomes. The platform also integrates with 
existing LMS (Learning Management Systems) to seamlessly incorporate data into teachers’ workflows. Furthermore, 
DeweyLearn offers detailed reports and visual analytics, allowing educators to identify areas for improvement 
and track progress over time. With a focus on K-12 education, DeweyLearn aims to drive continuous improvement 
in both teaching practices and student performance through data-driven insights. The developer who invented and 
created DeweyLearn is Dirk Liebich. The platform work was started in July of 2023 and has since grown to significantly.
"""

# Ingest the test document into the R2R system
res = app.ingest_documents(
    [
        Document(
            type="txt",
            data=test_string,
            metadata=meta,
        )
    ]
)

print(res)


In [24]:
# Ingest the test document into the R2R system
app.ingest_documents(
    [
        Document(
            type="txt",
            data="Just a dummy doc without owner.",
            metadata={},
        )
    ]
)

print(res)

{'processed_documents': ["Document 'Dirks test text' processed successfully."], 'failed_documents': [], 'skipped_documents': []}


#### SEARCH

In [12]:
from r2r import R2RClient
import json

# we are using a different port for DeweyLearn
client = R2RClient(base_url="http://localhost:9311")

filtered_search_response = client.search(
    "Who is Dirk Liebich?",
    use_vector_search=True,
    search_filters={"user_id": "2bc1f0e5-89ee-4e4b-be75-cdcb22144efb"},
    search_limit=2
)

print(json.dumps(filtered_search_response, indent=4))


{
    "results": {
        "vector_search_results": [
            {
                "id": "eef5da9b-1489-5ffb-9f71-081b20eab091",
                "score": 0.48747138567013004,
                "metadata": {
                    "text": "in both teaching practices and student performance through data-driven insights. The developer who invented and \ncreated DeweyLearn is Dirk Liebich. The platform work was started in July of 2023 and has since grown to significantly.",
                    "user": "dliebich",
                    "title": "Dirks test text",
                    "entity": "DeweyLearn",
                    "user_id": "2bc1f0e5-89ee-4e4b-be75-cdcb22144efb",
                    "version": "v0",
                    "chunk_order": 2,
                    "document_id": "1dc355b6-b2df-5720-b779-49864faf8ff8",
                    "extraction_id": "70135e63-d9a6-5854-8bf6-70daa5708ea5",
                    "associatedQuery": "Who is Dirk Liebich?"
                }
            },
    

In [13]:
from r2r import R2RClient

# we are using a different port for DeweyLearn
client = R2RClient(base_url="http://localhost:9311")

hyde_response = client.rag(
    "What are the main themes in Shakespeare's plays?",
    vector_search_settings={
        "search_strategy": "hyde",
        "search_limit": 10
    }
)

print('hyde_response = ', hyde_response)


TypeError: R2RClient.rag() got an unexpected keyword argument 'vector_search_settings'

In [17]:
hybrid_search_response = client.search(
    "Who is Dirk Liebich?",
    use_vector_search=True,
    search_limit=20,
    do_hybrid_search=True,
    search_filters={  # If you have specific filters, otherwise leave empty
        "hybrid_search_settings": {
            "full_text_weight": 1.0,
            "semantic_weight": 10.0,
            "full_text_limit": 200,
            "rrf_k": 25,
        }
    }
)

print(json.dumps(hybrid_search_response, indent=4))


{
    "results": {
        "vector_search_results": [],
        "kg_search_results": []
    }
}


In [26]:
# get all documents
documents_overview = client.documents_overview()
print(json.dumps(documents_overview, indent=4))

# get all documents by user_id
documents_overview = client.documents_overview(user_ids=['2bc1f0e5-89ee-4e4b-be75-cdcb22144efb'])
print(json.dumps(documents_overview, indent=4))



{
    "results": [
        {
            "document_id": "1dc355b6-b2df-5720-b779-49864faf8ff8",
            "version": "v0",
            "size_in_bytes": 1110,
            "metadata": {
                "user": "dliebich",
                "title": "Dirks test text",
                "entity": "DeweyLearn",
                "user_id": "2bc1f0e5-89ee-4e4b-be75-cdcb22144efb"
            },
            "status": "success",
            "user_id": "2bc1f0e5-89ee-4e4b-be75-cdcb22144efb",
            "title": "Dirks test text",
            "created_at": "2024-09-16T21:54:55.775412Z",
            "updated_at": "2024-09-16T21:54:55.775412Z"
        },
        {
            "document_id": "d18ac9f6-cb79-5c7c-98e8-42c7aeadfadf",
            "version": "v0",
            "size_in_bytes": 31,
            "metadata": {
                "title": "N/A"
            },
            "status": "success",
            "user_id": null,
            "title": "N/A",
            "created_at": "2024-09-16T22:35:04.36380

#### RAG

In [34]:

rag_response = client.rag(
    "Who is Dirk Liebich",
    use_vector_search=True,
    search_filters={"user_id": "2bc1f0e5-89ee-4e4b-be75-cdcb22144efb"},
    search_limit=2, 
)

print(rag_response['results']['completion']['choices'][0]['message']['content'])
print('')
print(json.dumps(rag_response, indent=4))


Dirk Liebich is the developer who invented and created DeweyLearn, an innovative AI-powered platform designed to assist teachers in analyzing classroom data [1].

{
    "results": {
        "completion": {
            "id": "chatcmpl-A8EkFMhnUxPo25tVhDClecMxVO13X",
            "choices": [
                {
                    "finish_reason": "stop",
                    "index": 0,
                    "message": {
                        "content": "Dirk Liebich is the developer who invented and created DeweyLearn, an innovative AI-powered platform designed to assist teachers in analyzing classroom data [1].",
                        "role": "assistant"
                    }
                }
            ],
            "created": 1726526667,
            "model": "gpt-4o-2024-05-13",
            "object": "chat.completion",
            "system_fingerprint": "fp_25624ae3a5",
            "usage": {
                "completion_tokens": 31,
                "prompt_tokens": 261,
           

In [4]:
from r2r import R2RClient
import json

# we are using a different port for DeweyLearn
client = R2RClient(base_url="http://localhost:9311")

response = client.rag(
    "What are the latest advancements in quantum computing?",
    rag_generation_config={
        "stream": False,
        "model": "openai/gpt-4o-mini",
        "temperature": 0.7,
        "max_tokens": 150
    }
)
print(json.dumps(response, indent=4))

{
    "results": {
        "completion": {
            "id": "chatcmpl-A8rgDnK2CUKnji1rUekbUNK8Xv3TB",
            "choices": [
                {
                    "finish_reason": "stop",
                    "index": 0,
                    "message": {
                        "content": "The provided context does not contain any information regarding advancements in quantum computing. It primarily discusses an AI-powered educational platform called DeweyLearn and its features related to teaching practices and student performance [1][2][3][4]. Therefore, I cannot provide an answer to the query about the latest advancements in quantum computing based on the given context.",
                        "role": "assistant"
                    }
                }
            ],
            "created": 1726676333,
            "model": "gpt-4o-mini-2024-07-18",
            "object": "chat.completion",
            "system_fingerprint": "fp_483d39d857",
            "usage": {
                "com

In [ ]:


# Sample questions for testing
questions_answers = [
    {"question": "What is the purpose of DeweyLearn?"},
    {"question": "How does DeweyLearn process classroom videos?"},
    {"question": "What kind of insights does DeweyLearn generate?"},
    {"question": "How does DeweyLearn integrate into teachers' workflows?"},
    {"question": "What are the main educational levels targeted by DeweyLearn?"},
    {"question": "How does DeweyLearn help improve teaching practices?"},
    {"question": "What kind of reports and analytics does DeweyLearn provide?"}
]

# Function to test RAG with the provided questions
def test_rag_function(app, questions_answers):
    results = []
    for qa in questions_answers:
        question = qa["question"]
        rag_results = app.rag(
            question, GenerationConfig(model="gpt-4o-mini", temperature=0.0)
        )
        result = {
            "question": question,
            "search_results": rag_results.search_results,
            "completion": rag_results.completion
        }
        results.append(result)
    return results

# Run test on RAG function
rag_test_results = test_rag_function(app, questions_answers)

# Print test results
for result in rag_test_results:
    print(f"Question: {result['question']}")
    print(f"Search Results:\n{result['search_results']}")
    print(f"Completion:\n{result['completion']}\n")
